In [90]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cyber-bullying-data-for-multi-label-classification/Data_Distribution.png
/kaggle/input/cyber-bullying-data-for-multi-label-classification/final_hateXplain.csv
/kaggle/input/cyber-bullying-data-for-multi-label-classification/hateXplain.csv


***Работаем с датасетом HateXplain: A Benchmark Dataset for Explainable Hate Speech Detection из Proceedings of the AAAI Conference on Artificial Intelligence, 2021. ***

С этим датасетом уже есть проект, который делит комментарии на normal/offensive/hatespeech с точностью 93%. Захотелось попробовать сделать классификацию по tergetам комментариев, насколько это возможно.


In [102]:
final_hate = pd.read_csv('/kaggle/input/cyber-bullying-data-for-multi-label-classification/final_hateXplain.csv')

In [103]:
final_hate

,comment,label,Race,Religion,Gender,Sexual Orientation,Miscellaneous
0,0 u0 lmao wow fuck you too 😂 😂,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
1,1 0 th floor maybe wow cnn with the fakenews t...,offensive,No_race,Nonreligious,No_gender,No_orientation,Other
2,1 0 yrs <number> white women raped by niggers ...,hatespeech,African,Nonreligious,No_gender,No_orientation,NaN
3,1 2 h ago ching chong accepted your friend req...,offensive,Asian,Nonreligious,No_gender,No_orientation,NaN
4,1 8 th century mayhem and lawlessness had noth...,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
...,...,...,...,...,...,...,...
20104,😂 😂 😂 😂 😂 y ’ all be so mad at caucasians fail...,offensive,African,Nonreligious,No_gender,No_orientation,NaN
20105,🤔 🤔 🤔 🤔 mmm yes the trebuchet memes are much n...,hatespeech,Arab,Islam,No_gender,No_orientation,NaN
20106,🤣 this so ghetto i ’ m ready to get tf,offensive,No_race,Nonreligious,No_gender,No_orientation,Other
20107,🤣 🤣 😂 😂 🤣 🤣 😂 😂 🤣 🤣 you spelled my name wrong ...,hatespeech,Arab,Nonreligious,No_gender,No_orientation,NaN


Посмотрим на все категории targetов:

In [105]:
print(final_hate['label'].value_counts())
print(final_hate['Race'].value_counts())
print(final_hate['Religion'].value_counts())
print(final_hate['Gender'].value_counts())
print(final_hate['Sexual Orientation'].value_counts())
print(final_hate['Miscellaneous'].value_counts())

label
normal        7818
hatespeech    6484
offensive     5807
Name: count, dtype: int64
Race
No_race       13579
African        3769
Arab           1172
Caucasian       802
Asian           400
Hispanic        313
Indigenous       42
Indian           32
Name: count, dtype: int64
Religion
Nonreligious    15387
Islam            2559
Jewish           1950
Christian         163
Hindu              42
Buddhism            8
Name: count, dtype: int64
Gender
No_gender    16380
Women         2262
Men           1467
Name: count, dtype: int64
Sexual Orientation
No_orientation    17827
Homosexual         2163
Heterosexual        113
Asexual               4
Bisexual              2
Name: count, dtype: int64
Miscellaneous
Other         2017
Refugee       1274
Minority       107
Disability      96
Economic        39
Name: count, dtype: int64


В соответствии с этим поменяем labels, чтобы было чёткое деление на виды комментариев:

In [118]:
new_labels = [
    'normal',
    'offensive', 'offensive_race', 'offensive_religion', 'offensive_gender', 'offensive_sexual_orientation', 'offensive_miscellaneous',
    'hatespeech','hatespeech_race', 'hatespeech_religion', 'hatespeech_gender', 'hatespeech_sexual_orientation', 'hatespeech_miscellaneous'
]

label_mapping = {
    'normal': 'normal',
    'offensive': ['offensive_race', 'offensive_religion', 'offensive_gender', 'offensive_sexual_orientation', 'offensive_miscellaneous'],
    'hatespeech': ['hatespeech_race', 'hatespeech_religion', 'hatespeech_gender', 'hatespeech_sexual_orientation', 'hatespeech_miscellaneous']
}
for i, row in final_hate.iterrows():
    for target, new_target_labels in label_mapping.items():
        if row['label'] == target:
            if 'African' in str(row['Race']):
                final_hate.at[i, 'label'] = 'offensive_race' if target == 'offensive' else 'hatespeech_race'
            elif 'Arab' in str(row['Race']):
                final_hate.at[i, 'label'] = 'offensive_race' if target == 'offensive' else 'hatespeech_race'
            elif 'Caucasian' in str(row['Race']):
                final_hate.at[i, 'label'] = 'offensive_race' if target == 'offensive' else 'hatespeech_race'
            elif 'Asian' in str(row['Race']):
                final_hate.at[i, 'label'] = 'offensive_race' if target == 'offensive' else 'hatespeech_race'
            elif 'Hispanic' in str(row['Race']):
                final_hate.at[i, 'label'] = 'offensive_race' if target == 'offensive' else 'hatespeech_race'
            elif 'Indigenous' in str(row['Race']):
                final_hate.at[i, 'label'] = 'offensive_race' if target == 'offensive' else 'hatespeech_race'
            elif 'Indian' in str(row['Race']):
                final_hate.at[i, 'label'] = 'offensive_race' if target == 'offensive' else 'hatespeech_race'
                
            elif 'Islam' in str(row['Religion']):
                final_hate.at[i, 'label'] = 'offensive_religion' if target == 'offensive' else 'hatespeech_religion'
            elif 'Jewish' in str(row['Religion']):
                final_hate.at[i, 'label'] = 'offensive_religion' if target == 'offensive' else 'hatespeech_religion'
            elif 'Christian' in str(row['Religion']):
                final_hate.at[i, 'label'] = 'offensive_religion' if target == 'offensive' else 'hatespeech_religion'
            elif 'Hindu' in str(row['Religion']):
                final_hate.at[i, 'label'] = 'offensive_religion' if target == 'offensive' else 'hatespeech_religion'
            elif 'Buddhism' in str(row['Religion']):
                final_hate.at[i, 'label'] = 'offensive_religion' if target == 'offensive' else 'hatespeech_religion'
                
            elif 'Women' in str(row['Gender']):
                final_hate.at[i, 'label'] = 'offensive_gender' if target == 'offensive' else 'hatespeech_gender'
            elif 'Men' in str(row['Gender']):
                final_hate.at[i, 'label'] = 'offensive_gender' if target == 'offensive' else 'hatespeech_gender'
                
            elif 'Homosexual' in str(row['Sexual Orientation']):
                final_hate.at[i, 'label'] = 'offensive_sexual_orientation' if target == 'offensive' else 'hatespeech_sexual_orientation'
            elif 'Heterosexual' in str(row['Sexual Orientation']):
                final_hate.at[i, 'label'] = 'offensive_sexual_orientation' if target == 'offensive' else 'hatespeech_sexual_orientation'
            elif 'Asexual' in str(row['Sexual Orientation']):
                final_hate.at[i, 'label'] = 'offensive_sexual_orientation' if target == 'offensive' else 'hatespeech_sexual_orientation'
            elif 'Bisexual' in str(row['Sexual Orientation']):
                final_hate.at[i, 'label'] = 'offensive_sexual_orientation' if target == 'offensive' else 'hatespeech_sexual_orientation'
                
            elif 'Other' in str(row['Miscellaneous']):
                final_hate.at[i, 'label'] = 'offensive_miscellaneous' if target == 'offensive' else 'hatespeech_miscellaneous'
            elif 'Refugee' in str(row['Miscellaneous']):
                final_hate.at[i, 'label'] = 'offensive_miscellaneous' if target == 'offensive' else 'hatespeech_miscellaneous'
            elif 'Minority' in str(row['Miscellaneous']):
                final_hate.at[i, 'label'] = 'offensive_miscellaneous' if target == 'offensive' else 'hatespeech_miscellaneous'
            elif 'Disability' in str(row['Miscellaneous']):
                final_hate.at[i, 'label'] = 'offensive_miscellaneous' if target == 'offensive' else 'hatespeech_miscellaneous'
            elif 'Economic' in str(row['Miscellaneous']):
                final_hate.at[i, 'label'] = 'offensive_miscellaneous' if target == 'offensive' else 'hatespeech_miscellaneous'
                
            else:
                final_hate.at[i, 'label'] = target

In [107]:
print(final_hate['label'].value_counts())

label
normal                           5392
hatespeech_race                  4615
hatespeech_religion              2194
offensive_race                   1915
offensive_gender                 1038
hatespeech_gender                 875
offensive_religion                816
offensive_miscellaneous           812
offensive                         694
hatespeech_miscellaneous          680
offensive_sexual_orientation      532
hatespeech_sexual_orientation     495
hatespeech                         51
Name: count, dtype: int64


In [110]:
final_hate.rename(mapper={'comment': 'post_tokens'}, inplace=True, axis=1)

In [111]:
final_hate

,post_tokens,label,Race,Religion,Gender,Sexual Orientation,Miscellaneous
0,0 u0 lmao wow fuck you too 😂 😂,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
1,1 0 th floor maybe wow cnn with the fakenews t...,offensive_miscellaneous,No_race,Nonreligious,No_gender,No_orientation,Other
2,1 0 yrs <number> white women raped by niggers ...,hatespeech_race,African,Nonreligious,No_gender,No_orientation,NaN
3,1 2 h ago ching chong accepted your friend req...,offensive_race,Asian,Nonreligious,No_gender,No_orientation,NaN
4,1 8 th century mayhem and lawlessness had noth...,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
...,...,...,...,...,...,...,...
20104,😂 😂 😂 😂 😂 y ’ all be so mad at caucasians fail...,offensive_race,African,Nonreligious,No_gender,No_orientation,NaN
20105,🤔 🤔 🤔 🤔 mmm yes the trebuchet memes are much n...,hatespeech_race,Arab,Islam,No_gender,No_orientation,NaN
20106,🤣 this so ghetto i ’ m ready to get tf,offensive_miscellaneous,No_race,Nonreligious,No_gender,No_orientation,Other
20107,🤣 🤣 😂 😂 🤣 🤣 😂 😂 🤣 🤣 you spelled my name wrong ...,hatespeech_race,Arab,Nonreligious,No_gender,No_orientation,NaN


In [114]:
hate_train = final_hate.sample(frac=0.7, random_state=42)
hate_test = final_hate.drop(hate_train.index)

In [115]:
hate_train

,post_tokens,label,Race,Religion,Gender,Sexual Orientation,Miscellaneous
1337,<user> <user> the shoe being on the other foot...,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
928,<user> <user> i mean sure he raped women publi...,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
15487,so who is the biggest dhimmi moslem cock sucke...,hatespeech_religion,No_race,Islam,No_gender,No_orientation,NaN
4043,<user> this is a win for the dykes,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
16317,the cross is superior to the inferior crescent...,hatespeech_religion,No_race,Islam,No_gender,No_orientation,NaN
...,...,...,...,...,...,...,...
11646,jangan merasa superior cuma gara gara lo boygr...,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
10570,i ’ ve always wanted to beat up a nigga in fro...,hatespeech_race,African,Nonreligious,Men,No_orientation,NaN
15461,so tired of all this gay shit,offensive_sexual_orientation,No_race,Nonreligious,No_gender,Homosexual,Other
14344,perrie really loves asians uno,normal,No_race,Nonreligious,No_gender,No_orientation,NaN


In [116]:
hate_test

,post_tokens,label,Race,Religion,Gender,Sexual Orientation,Miscellaneous
9,2 3 andme is a strange choice for this sort of...,hatespeech_religion,No_race,Jewish,No_gender,No_orientation,NaN
11,2 v2 challs kroc is scared of comp so dont be ...,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
13,3 novices cm jagan to inaugurate milli council...,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
16,4 ft tall beaner women still dispensing <numbe...,offensive_race,Hispanic,Nonreligious,Women,No_orientation,Disability
24,<money> says she a an overweight dyke with blu...,hatespeech_sexual_orientation,No_race,Nonreligious,No_gender,Homosexual,NaN
...,...,...,...,...,...,...,...
20095,🗣 stop crying over that manipulative toxic ass...,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
20101,😂 😂 😂 i could just imagine at least i know not...,hatespeech_religion,No_race,Islam,No_gender,No_orientation,NaN
20103,😂 😂 😂 😂 😂 defending sand niggers 😂 😂 😂 😂 😂 you...,offensive_race,African,Islam,No_gender,No_orientation,NaN
20106,🤣 this so ghetto i ’ m ready to get tf,offensive_miscellaneous,No_race,Nonreligious,No_gender,No_orientation,Other


In [81]:
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder

In [119]:
def get_features(cv, tfidf, x):
    
    cv_feature = cv.transform(x['post_tokens'])
    tfidf_feature = tfidf.transform(x['post_tokens'])
    
    return (cv_feature + tfidf_feature) / 2

def get_labels(oe, x):
    y = oe.transform(x[['label']])
    
    return y

cv = CountVectorizer(tokenizer = word_tokenize, ngram_range = (1, 2))
tfidf = TfidfVectorizer(tokenizer = word_tokenize, ngram_range = (1, 2))
oe = OrdinalEncoder(categories = [new_labels])

cv.fit(hate_train['post_tokens'])
tfidf.fit(hate_train['post_tokens'])
oe.fit(hate_train[['label']])

x_train = get_features(cv, tfidf, hate_train)
y_train = get_labels(oe, hate_train)
x_test = get_features(cv, tfidf, hate_test)
y_test = get_labels(oe, hate_test)

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Теперь попробуем разные методы классификации:

In [157]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)
lr_score = lr.score(x_test, y_test)
print(f"Точность логистической регрессии: {lr_score:.2f}")

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Точность логистической регрессии: 0.49


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [160]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)
dt_score = dt.score(x_test, y_test)
print(f"Точность дерева решений: {dt_score:.2f}")

Точность дерева решений: 0.40


In [161]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf_score = rf.score(x_test, y_test)
print(f"Точность случайного леса: {rf_score:.2f}")


/tmp/ipykernel_33/3409715518.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train, y_train)


Точность случайного леса: 0.47


In [162]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(x_train, y_train)
gb_score = gb.score(x_test, y_test)
print(f"Точность градиентного бустинга: {gb_score:.2f}")


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

KeyboardInterrupt



In [ ]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(x_train, y_train)
svm_score = svm.score(x_test, y_test)
print(f"Точность SVM: {svm_score:.2f}")


Ну и попробуем MLPClassifier (с ним точность классификации на обычные normal/offensive/hate - 93%)

In [136]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state=0, verbose=True, batch_size=1000, max_iter=30, warm_start=True)
mlp.fit(x_train, y_train.ravel())

Iteration 1, loss = 2.40938739
Iteration 2, loss = 1.90928593
Iteration 3, loss = 1.49927667
Iteration 4, loss = 1.09734071
Iteration 5, loss = 0.75185647
Iteration 6, loss = 0.51442342
Iteration 7, loss = 0.36157243
Iteration 8, loss = 0.26389915
Iteration 9, loss = 0.19993100
Iteration 10, loss = 0.15622266
Iteration 11, loss = 0.12529999
Iteration 12, loss = 0.10302627
Iteration 13, loss = 0.08591568
Iteration 14, loss = 0.07269899
Iteration 15, loss = 0.06233541
Iteration 16, loss = 0.05406329
Iteration 17, loss = 0.04730780
Iteration 18, loss = 0.04176942
Iteration 19, loss = 0.03820446
Iteration 20, loss = 0.03436534
Iteration 21, loss = 0.03087857
Iteration 22, loss = 0.02795072
Iteration 23, loss = 0.02546370
Iteration 24, loss = 0.02328279
Iteration 25, loss = 0.02135352
Iteration 26, loss = 0.01969185
Iteration 27, loss = 0.01811900
Iteration 28, loss = 0.01687668
Iteration 29, loss = 0.01573433
Iteration 30, loss = 0.01471410


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(batch_size=1000, max_iter=30, random_state=0, verbose=True,
              warm_start=True)

In [137]:
from sklearn.metrics import classification_report

y_pred = mlp.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.48      0.70      0.57      1621
         1.0       0.29      0.15      0.19       213
         2.0       0.28      0.12      0.16       579
         3.0       0.31      0.06      0.10       244
         4.0       0.43      0.24      0.31       314
         5.0       0.38      0.29      0.33       150
         6.0       0.21      0.13      0.16       247
         7.0       0.00      0.00      0.00        17
         8.0       0.53      0.69      0.60      1349
         9.0       0.50      0.59      0.54       698
        10.0       0.15      0.07      0.10       253
        11.0       0.35      0.13      0.19       141
        12.0       0.21      0.08      0.11       207

    accuracy                           0.46      6033
   macro avg       0.32      0.25      0.26      6033
weighted avg       0.42      0.46      0.42      6033



Получили уже привычные 0,46 - в целом, понятно, что для получения большей точности нужно примерно одинаковое распределение высказываний всех типов и другой подход, но это было интересно:)